# Graph Practice III #

In [2]:
from AdjListGraph import Graph
from AdjListGraph import Vertex

### <a id='Ex1'>Ex.1 Is Graph Bipartite?</a>

Given an undirected graph, return true if and only if it is bipartite.

Recall that a graph is bipartite if we can split it's set of nodes into two independent subsets A and B such that every edge in the graph has one node in A and another node in B.

The graph is given in the following form: graph[i] is a list of indexes j for which the edge between nodes i and j exists.  Each node is an integer between 0 and graph.length - 1.  There are no self edges or parallel edges: graph[i] does not contain i, and it doesn't contain any element twice.

<img src="../images/ch17/bipartite1.png" width="440"/>
<img src="../images/ch17/bipartite2.png" width="540"/>

** Solution **

Our goal is trying to use two colors to color the graph and see if there are any adjacent nodes having the same color.
Initialize a color[] array for each node. Here are three states for colors[] array:

-1: Haven't been colored.

0: Blue.

1: Red.

For each node,

If it hasn’t been colored, use a color to color it. Then use the other color to color all its adjacent nodes (DFS).
If it has been colored, check if the current color is the same as the color that is going to be used to color it. 

In [6]:
def isBipartite(graph):
    color = {} # key是node的 ID,  value是 颜色: 0 1
    def dfs(pos):
        for i in graph[pos]: # 对于每个node
            if i in color: # i是邻居  color[i]是邻居的color    color[pos]是当前node的颜色 
                if color[i] == color[pos]: return False
            else:
                color[i] = color[pos] ^ 1 # 颜色反转
                if not dfs(i): return False  # 递归调用
        return True
    
    for i in range(len(graph)): # 对每个node进行check
        if i not in color: color[i] = 0 # 未访问过, 颜色设置成0
        if not dfs(i): return False # 如果dfs运行不下去 是False的话, 就返回False
    return True

In [7]:
graph = [[1,3], [0,2], [1,3], [0,2]]
isBipartite(graph)

True

In [8]:
graph = [[1,2,3], [0,2], [0,1,3], [0,2]]
isBipartite(graph)

False

### <a id='Ex2'>Ex.2 Pacific Atlantic Water Flow</a>

Given an m x n matrix of non-negative integers representing the height of each unit cell in a continent, the "Pacific ocean" touches the left and top edges of the matrix and the "Atlantic ocean" touches the right and bottom edges.

Water can only flow in four directions (up, down, left, or right) from a cell to another one with height equal or lower.

Find the list of grid coordinates where water can flow to both the Pacific and Atlantic ocean.

<img src="../images/ch17/ocean.png" width="740"/>

In [9]:
def pacificAtlantic(matrix):

    if not matrix: return []
    directions = [(1,0),(-1,0),(0,1),(0,-1)]
    m = len(matrix)
    n = len(matrix[0])
    p_visited = [[False for _ in range(n)] for _ in range(m)]

    a_visited = [[False for _ in range(n)] for _ in range(m)]
    result = []

    for i in range(m):
        # p_visited[i][0] = True
        # a_visited[i][n-1] = True
        dfs(matrix, i, 0, p_visited, m, n)   # 最左边
        dfs(matrix, i, n-1, a_visited, m, n) # 最右边
    for j in range(n):
        # p_visited[0][j] = True
        # a_visited[m-1][j] = True
        dfs(matrix, 0, j, p_visited, m, n)  # 最上面
        dfs(matrix, m-1, j, a_visited, m, n)# 最下面

    for i in range(m):
        for j in range(n):
            if p_visited[i][j] and a_visited[i][j]: # 求交集  都是True的点
                result.append([i,j])
    return result


def dfs(matrix, i, j, visited, m, n):
    # when dfs called, meaning its caller already verified this point 
    visited[i][j] = True
    for dir in [(1,0),(-1,0),(0,1),(0,-1)]:
        x, y = i + dir[0], j + dir[1]
        if x < 0 or x >= m or y < 0 or y >= n or visited[x][y] or matrix[x][y] < matrix[i][j]: # 下面找的点要大于等于我 才能继续找,小于我就不能反向推了
            continue
        dfs(matrix, x, y, visited, m, n)

In [10]:
matrix = [
    [1,2,2,3,4],
    [3,2,3,4,4],
    [2,4,5,3,1],
    [6,7,1,4,5],
    [5,1,1,2,4]
]
pacificAtlantic(matrix)

[[0, 4], [1, 3], [1, 4], [2, 2], [3, 0], [3, 1], [4, 0]]

In [14]:
from collections import deque
def pacificAtlantic(matrix):
    if not matrix: return []
    m, n = len(matrix), len(matrix[0])
    def bfs(reachable_ocean):
        q = deque(reachable_ocean)
        while q: # q是deque 先进先出
            (i, j) = q.popleft()
            for (di, dj) in [(0,1), (0, -1), (1, 0), (-1, 0)]:
                if 0 <= di+i < m and 0 <= dj+j < n and (di+i, dj+j) not in reachable_ocean \
                    and matrix[di+i][dj+j] >= matrix[i][j]:
                    q.append( (di+i,dj+j) )
                    reachable_ocean.add( (di+i, dj+j) )
        return reachable_ocean         
    pacific  =set ( [ (i, 0) for i in range(m)]   + [(0, j) for j  in range(1, n)]) #把 左边 和 上边 的点放进set
    atlantic =set ( [ (i, n-1) for i in range(m)] + [(m-1, j) for j in range(n-1)]) #把 右边 和 下边 的点放进set
    return list( bfs(pacific) & bfs(atlantic) )

In [15]:
pacificAtlantic(matrix)

[(1, 3), (3, 0), (3, 1), (1, 4), (0, 4), (2, 2), (4, 0)]

### <a id='Ex3'>Ex.3 Longest Increasing Path in a Matrix</a>

Given an integer matrix, find the length of the longest increasing path.

From each cell, you can either move to four directions: left, right, up or down. You may NOT move diagonally or move outside of the boundary (i.e. wrap-around is not allowed).

<img src="../images/ch17/longest1.png" width="100"/>
<img src="../images/ch17/longest2.png" width="100"/>

In [11]:
def longestIncreasingPath(matrix):
    if not matrix: return 0
    directions = [(1,0),(-1,0),(0,1),(0,-1)]
    m = len(matrix)
    n = len(matrix[0])
    cache = [[-1 for _ in range(n)] for _ in range(m)] #存放答案 当前的每个点,以这个点为起始点能组成的最长的递增的数组
    res = 0
    for i in range(m): 
        for j in range(n):
            cur_len = dfs(i, j, matrix, cache, m, n)
            res = max(res, cur_len)
    return res

def dfs(i, j, matrix, cache, m, n):
    if cache[i][j] != -1:
        return cache[i][j] # 访问过了  查询一下 
    res = 1
    for direction in [(1,0),(-1,0),(0,1),(0,-1)]:
        x, y = i + direction[0], j + direction[1]
        if x < 0 or x >= m or y < 0 or y >= n or matrix[x][y] <= matrix[i][j]: # 没有越界 且是 递增的,不是递增的就continue
            continue
        length = 1 + dfs(x, y, matrix, cache, m, n) # 当前长度 + 邻居的长度
        res = max(length, res)
    cache[i][j] = res
    return res

In [12]:
nums = [
  [9,9,4],
  [6,6,8],
  [2,1,1]
]
longestIncreasingPath(nums)

4

In [13]:
nums = [
  [8,4,5],
  [3,9,6],
  [2,8,7]
]
longestIncreasingPath(nums)

6

### <a id='Ex4'>Ex.4 01 Matrix</a>

Given a matrix consists of 0 and 1, find the distance of the nearest 0 for each cell.

The distance between two adjacent cells is 1.

Example 1: 

Input:

0 0 0

0 1 0

0 0 0

Output:

0 0 0

0 1 0

0 0 0

Example 2: 

Input:

0 0 0

0 1 0

1 1 1

Output:

0 0 0

0 1 0

1 2 1

In [20]:
def updateMatrix(matrix):
    q, m, n = [], len(matrix), len(matrix[0])
    for i in range(m):
        for j in range(n):
            if matrix[i][j] != 0:
                matrix[i][j] = 0x7fffffff # 非0的地方设置成最大值 因为要找最小值
            else:
                q.append((i, j)) # 把所有的0放进stack
    for i, j in q:
        for r, c in ((i, 1+j), (i, j-1), (i+1, j), (i-1, j)): # 移动后的位置  (r,c)
            z = matrix[i][j] + 1
            if 0 <= r < m and 0 <= c < n and matrix[r][c] > z: # 找 neighbor
                matrix[r][c] = z
                q.append((r, c)) # 上面q放的0  现在放的 >z的 位置
    return matrix

In [21]:
matrix = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
]
updateMatrix(matrix)

[[0, 0, 0], [0, 1, 0], [0, 0, 0]]

In [22]:
matrix = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 1, 1],
]
updateMatrix(matrix)

[[0, 0, 0], [0, 1, 0], [0, 1, 1]]

In [8]:
def updateMatrix2(matrix):
    def DP(i, j, m, n, dp):
        if i > 0: dp[i][j] = min(dp[i][j], dp[i - 1][j] + 1)
        if j > 0: dp[i][j] = min(dp[i][j], dp[i][j - 1] + 1)
        if i < m - 1: dp[i][j] = min(dp[i][j], dp[i + 1][j] + 1)
        if j < n - 1: dp[i][j] = min(dp[i][j], dp[i][j + 1] + 1)
            
    if not matrix: return [[]]
    m, n = len(matrix), len(matrix[0])
    dp = [[0x7fffffff if matrix[i][j] != 0 else 0 for j in range(n)] for i in range(m)]
    for i in range(m):
        for j in range(n):
            DP(i, j, m, n, dp)

    for i in range(m - 1, -1, -1):
        for j in range(n - 1, -1, -1):
            DP(i, j, m, n, dp)

    return dp



In [9]:
matrix = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
]
updateMatrix2(matrix)

[[0, 0, 0], [0, 1, 0], [0, 0, 0]]

In [10]:
matrix = [
    [0, 0, 0],
    [0, 1, 0],
    [1, 1, 1],
]
updateMatrix2(matrix)

[[0, 0, 0], [0, 1, 0], [1, 2, 1]]